In [ ]:
import pickle
from collections import namedtuple

import os
from urllib.parse import parse_qs

import ipywidgets as widgets
import ipyvuetify as v
import traitlets

Peiling = namedtuple('Peiling', ['verwacht', 'laag', 'hoog'])

In [ ]:
with open('peilingen.pkl', 'rb') as fh:
    numbers = pickle.load(fh)
with open('table.pkl', 'rb') as fh:
    table = pickle.load(fh)

parties = list(numbers.keys())

In [ ]:
query_string = os.environ.get('QUERY_STRING', '')
server = os.environ.get('SERVER_NAME', 'localhost')
base_url = "http://" + server
port = os.environ.get('SERVER_PORT', '')
if port:
    base_url += ":" + port
base_url += os.environ.get('SCRIPT_NAME', '')

parties_in_query = parse_qs(query_string).get('parties', [''])[0].split(sep=',')
url_picked_parties = tuple(set(parties_in_query) & set(parties))

# Coalitiewijzer

In [ ]:
class SeatsWidget(v.VuetifyTemplate):
    rows = traitlets.Int(5).tag(sync=True)
    cols = traitlets.Int(30).tag(sync=True)
    low = traitlets.Int(0).tag(sync=True)
    exp = traitlets.Int(0).tag(sync=True)
    high = traitlets.Int(0).tag(sync=True)
    template = traitlets.Unicode('''
<template>
    <div>
        <table>
        <tr v-for="i in rows">
            <td v-for="j in cols/2">
                <div>
                    <v-icon v-if="((j-1) * rows + i) < low"        style="color: #666666">mdi-seat</v-icon>
                    <v-icon v-else-if="((j-1) * rows + i) < exp"   style="color: red; opacity: 30%; transform: rotate(90deg)">mdi-seat</v-icon>
                    <v-icon v-else-if="((j-1) * rows + i) == exp"  style="color: red">mdi-seat</v-icon>
                    <v-icon v-else-if="((j-1) * rows + i) <= high" style="color: red; opacity: 30%; transform: rotate(-90deg)">mdi-seat</v-icon>
                    <v-icon v-else style="color: #bbbbbb; transform: rotate(180deg)">mdi-seat</v-icon>
                </div>
            </td>
            <td>
                <div>
                    <v-icon style="color: blue">mdi-dots-vertical</v-icon>
                </div>
            </td>
            <td v-for="j in cols/2">
                <div>
                    <v-icon v-if="((j + cols/2 - 1) * rows + i) < low"        style="color: #666666">mdi-seat</v-icon>
                    <v-icon v-else-if="((j + cols/2 - 1) * rows + i) < exp"   style="color: red; opacity: 30%; transform: rotate(90deg)">mdi-seat</v-icon>
                    <v-icon v-else-if="((j + cols/2 - 1) * rows + i) == exp"  style="color: red">mdi-seat</v-icon>
                    <v-icon v-else-if="((j + cols/2 - 1) * rows + i) <= high" style="color: red; opacity: 30%; transform: rotate(-90deg)">mdi-seat</v-icon>
                    <v-icon v-else style="color: #bbbbbb; transform: rotate(180deg)">mdi-seat</v-icon>
                </div>
            </td>
        </tr>
    </div>
</template>
''').tag(sync=True)

seats = SeatsWidget()

In [ ]:
url_picked_party_indices = [ix for ix, party in enumerate(parties) if party in url_picked_parties]
buttons = {party: v.Btn(text=True, children=[party]) for party in parties}

toggle_multi = v.BtnToggle(v_model=url_picked_party_indices, multiple=True, children=list(buttons.values()))

In [ ]:
url = base_url
if len(query_string) > 0:
    url += query_string

share_url = widgets.HTML(
    value=f'<a href="{url}">{url}</a>',
    description='Share link:',
)

In [ ]:
def on_toggle(btn_toggle, event, data):
    selected = tuple(parties[ix] for ix in sorted(data))
    if len(selected) == 1:
        peiling = numbers[selected[0]]
    else:
        peiling = table[selected]

    seats.low = peiling.laag
    seats.exp = peiling.verwacht
    seats.high = peiling.hoog
    
    url = base_url
    if len(selected) > 0:
        url = base_url + "?" + "parties=" + ",".join(selected)

    share_url.value = f'<a href="{url}">{url}</a>'

toggle_multi.on_event('change', on_toggle)
if len(url_picked_party_indices) > 0:
    on_toggle(None, None, url_picked_party_indices)

display(seats)
v.Layout(children=[toggle_multi])

Druk op de knoppen om partijen toe te voegen aan je coalitie. Een meerderheid heeft 76 zetels nodig. De helder rode zetel is het verwachte aantal zetels. In licht rood (zijwaarts gekantelde zetels) zie je onder- en bovengrenzen van het verwachte aantal zetels op basis van verschillende peilingen.

Press the buttons to add parties to your coalition. A majority coalition needs 76 seats. The bright red seat is the expected number of seats. In light red (tipped sideways seats) we show the lower and upper limits of the expected amount of seats based on different polls.

In [ ]:
display(share_url)

---

Data: [Peilingwijzer (op basis van peilingen I&O Research, Ipsos en Kantar)](https://peilingwijzer.tomlouwerse.nl/). Code: [see blogpost on Medium](https://blog.esciencecenter.nl/coalition-polls-for-the-people-with-coalitiewijzer-68bca83b95e7).

By [Patrick Bos](https://twitter.com/egpbos/)

[<img src="logo.png" alt="Netherlands eScience Center logo" style="height: 1.5em;" align="left"/>](https://esciencecenter.nl/)